In [2]:
import pandas as pd
import numpy as np
import re
import whois

In [3]:
base_df = pd.read_csv('/Users/chrismclearnon/Developer/QUB-CSC3032-FYP/src/data/dataset.csv')
print(base_df)

                                                     URL  URLType
0      http://mylust.com/videos/215137/my-raunchy-loo...        0
1      http://mic.com/articles/101766/one-perfect-twe...        0
2      http://stackexchange.com/newsletters/newslette...        0
3      http://bisaneema.org/%e0%a4%ad%e0%a4%be%e0%a4%...        1
4      http://skyrock.com/videos/eDJwODRhNzoxMw==-bug...        0
...                                                  ...      ...
46939  http://themeforest.net/item/artistas-modern-po...        0
46940  http://torcache.net/torrent/2EE5318B3923B273EF...        0
46941  http://grasslandhotel.com.vn/index.php/compone...        1
46942  http://conservativetribune.com/barack-obama-ju...        0
46943  http://sberbank.ru/moscow/ru/person/investment...        0

[46944 rows x 2 columns]


In [50]:
import datetime
from socket import *

def host_ft_extract(url):
    if not url:
        return
    
    feature_list = []
    url_dt = {}
    url_regex = '(?:http.*://)?(?P<host>[^:/ ]+).*'
    url_search = re.search(url_regex, url)
    hostname = url_search.group('host')

    try:
        domain = whois.whois(hostname)
    except whois.parser.PywhoisError:
        url_dt.update({
            "RegistryDate": "",
            "ExpirationDate": "",
            "HostCountry": "",
            "DomainAge": 0,
            "ExpYear": 0,
            "ExpMonth": 0,
            "ExpDay": 0
        })
        
        feature_list.append(url_dt)
        df = pd.DataFrame(feature_list)
        return df
    except timeout:
        url_dt.update({
            "RegistryDate": "",
            "ExpirationDate": "",
            "HostCountry": "",
            "DomainAge": 0,
            "ExpYear": 0,
            "ExpMonth": 0,
            "ExpDay": 0
        })
        
        feature_list.append(url_dt)
        df = pd.DataFrame(feature_list)
        
        return df
    except gaierror:
        url_dt.update({
            "RegistryDate": "",
            "ExpirationDate": "",
            "HostCountry": "",
            "DomainAge": 0,
            "ExpYear": 0,
            "ExpMonth": 0,
            "ExpDay": 0
        })
        
        feature_list.append(url_dt)
        df = pd.DataFrame(feature_list)
        
        return df
        
        
    reg_date = domain.creation_date
    
    if isinstance(reg_date, list):
        url_dt.update({"RegistryDate": reg_date[0]})
    else:
        url_dt.update({"RegistryDate": reg_date})

    exp_date = domain.expiration_date
    if isinstance(exp_date, list):
        url_dt.update({"ExpirationDate": exp_date[0]})
    else:
        url_dt.update({"ExpirationDate": exp_date})

    country = domain.country
    url_dt.update({"HostCountry": country})
    #print(url_dt)
    domain_age = datetime.datetime.now() - url_dt["RegistryDate"]
    age_in_days = domain_age.days
    url_dt.update({"DomainAge": age_in_days})

    feature_list.append(url_dt)
    df = pd.DataFrame(feature_list)
    df["ExpYear"] = df["ExpirationDate"].dt.year
    df["ExpMonth"] = df["ExpirationDate"].dt.month
    df["ExpDay"] = df["ExpirationDate"].dt.day
    
    return df

In [51]:
dataframe_list = []
for idx, row in base_df.iterrows():
    print(row["URL"])
    feature_df = host_ft_extract(row['URL'])
    feature_df['URLType'] = row["URLType"]
    dataframe_list.append(feature_df)
    
extracted_df = pd.DataFrame.concat(dataframe_list)
print(extracted_df)

http://mylust.com/videos/215137/my-raunchy-looking-wife-definitely-loves-missionary-position/
http://mic.com/articles/101766/one-perfect-tweet-calls-out-the-hypocrisy-of-how-the-media-talks-about-black-violence
http://stackexchange.com/newsletters/newsletter?site=productivity.stackexchange.com
http://bisaneema.org/%e0%a4%ad%e0%a4%be%e0%a4%97%e0%a4%b5%e0%a4%a4-%e0%a4%9c%e0%a5%8d%e0%a4%9e%e0%a4%be%e0%a4%a8-%e0%a4%af%e0%a4%9c%e0%a5%8d%e0%a4%9e-%e0%a4%b8%e0%a4%ae%e0%a4%bf%e0%a4%a4%e0%a4%bf/
http://skyrock.com/videos/eDJwODRhNzoxMw==-bug-1187-moi-ma-famille.html?top=3&period=recent
http://qz.com/404384/after-ten-days-on-the-mcdonalds-diet-my-sons-gut-bacteria-was-ravaged/
http://espn.go.com/blog/houston-rockets/post/_/id/755/rockets-adjustments-keeps-series-season-alive
http://tinnhanh360.net/tran-chien-solo-1-1-kinh-dien-nhat-trong-su-nghiep-scott-adkins.html
https://serverfault.com/questions/512828/virtual-interface-gets-static-ip-actual-interface-doesnt
http://tobogo.net/cdsb/board.php?b

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'NoneType'